In [1]:
import numpy as np
from DataGenerator import DatatGenerator

In [2]:
data = DatatGenerator()
data.connect()
data.setup_world()
data.setup_writer()
data.setup_cameras()

Connecting to CARLA at 172.31.32.1:2000
Connected to Carla/Maps/Town10HD_Opt
Vehicle spawned at Location(x=-15.407496, y=133.728470, z=0.600000)
Autopilot enabled
Camera 'front' spawned and recording to ./EK505-Data/front/
Camera 'back' spawned and recording to ./EK505-Data/back/
Camera 'left' spawned and recording to ./EK505-Data/left/
Camera 'right' spawned and recording to ./EK505-Data/right/


In [3]:
data.world.tick()

5190

In [4]:
camera_names = ["front", "back", "left", "right"]
cameras = data.cameras
cameras

In [5]:
# extrinsics_list = []
# for camera in cameras:
#     # extrinsics = np.array(camera.get_transform().get_matrix())
#     extrinsics = camera.get_transform().get_inverse_matrix()
#     extrinsics_list.append(extrinsics)
# # extrinsics_list

extrinsics_dict = dict.fromkeys(camera_names)
# for name, extrinsics in zip(camera_names, extrinsics_list):
#     extrinsics_dict[name] = extrinsics

for name, cam in zip(camera_names, cameras):
    extrinsics_dict[name] = np.array(cam.get_transform().get_inverse_matrix())

In [6]:
# Reference Sensor - Front
reference_sensor = 'front'

T_w_front = extrinsics_dict[reference_sensor]

calibrations = {}
for k in camera_names:
    w_to_k = extrinsics_dict[k]
    if k == reference_sensor:
        calibrations[k] = np.eye(4)
    else:
        calibrations[k] = w_to_k @ np.linalg.inv(T_w_front)

calibrations


{'front': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'back': array([[-9.84807755e-01,  3.39933676e-08, -1.73648163e-01,
         -1.21833962e+00],
        [-3.38632796e-08, -1.00000000e+00, -2.96265281e-09,
         -5.31854482e-07],
        [-1.73648165e-01,  2.96012819e-09,  9.84807755e-01,
         -1.06591048e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 'left': array([[ 7.59613451e-03, -9.96194708e-01, -8.68240802e-02,
         -8.22900806e-01],
        [ 9.96194690e-01,  1.19251700e-08,  8.71557343e-02,
          2.38383487e-01],
        [-8.68240815e-02, -8.71557372e-02,  9.92403878e-01,
         -7.19944073e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 'right': array([[ 7.59619724e-03,  9.96194708e-01, -8.68240747e-02,
         -8.22909781e-01],
        [-9.96194689e-01,  7.56174989e-08, -8.71557342e-02,
         -2.3

In [11]:
import json

calibrations_serializable = {
    name: mat.tolist() for name, mat in calibrations.items()
}

with open('./extrinsics.json', 'w') as f:
    json.dump(calibrations_serializable, f, indent=4)

In [8]:
def build_projection_matrix(w, h, fov, is_behind_camera=False):
    # https://blog.wuhanstudio.uk/blog/carla-coordinate/
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    if is_behind_camera:
        K[0, 0] = K[1, 1] = -focal
    else:
        K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

w = 1280
h = 720
fov = 90

build_projection_matrix(w, h, fov)

array([[640.,   0., 640.],
       [  0., 640., 360.],
       [  0.,   0.,   1.]])